In [6]:
import pandas as pd
import matplotlib.pyplot as plt
from cobra.io import read_sbml_model
model = read_sbml_model('iML1515.xml.gz')
model

Name,iML1515
Memory address,0x02586bf593d0
Number of metabolites,1877
Number of reactions,2712
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [7]:
model_sucd= model.copy()
model_sucd

Name,iML1515
Memory address,0x025861ebbe50
Number of metabolites,1877
Number of reactions,2712
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [8]:
from cobra import Reaction, Metabolite
sucd = Reaction('SUCD') #CoA-dependent succinate semialdehyde dehydrogenase
succinyl_semialdehyde = Metabolite(id='succinyl_semialdehyde_c', compartment='c')

sucd.add_metabolites({model_sucd.metabolites.h_c: 1,
                              model_sucd.metabolites.nadh_c: -1,
                              model_sucd.metabolites.succoa_c: -1,
                              succinyl_semialdehyde: 1, #new one
                              model_sucd.metabolites.coa_c: 1,
                              model_sucd.metabolites.nad_c: 1
                             })
print(sucd.build_reaction_string())
model_sucd.add_reactions([sucd])
model_sucd.reactions.SUCD

nadh_c + succoa_c --> coa_c + h_c + nad_c + succinyl_semialdehyde_c


Reaction identifier,SUCD
Name,
Memory address,0x02587739e310
Stoichiometry,nadh_c + succoa_c --> coa_c + h_c + nad_c + succinyl_semialdehyde_c Nicotinamide adenine dinucleotide - reduced + Succinyl-CoA --> Coenzyme A + H+ + Nicotinamide adenine dinucleotide +
GPR,
Lower bound,0.0
Upper bound,1000.0


In [9]:
from cobra import Reaction, Metabolite
fhbd = Reaction('FHBD') #4-hydroxybutyrate dehydrogenase
four_hydroxybutyrate = Metabolite(id='four_hydroxybutyrate_c', compartment='c')

fhbd.add_metabolites({model_sucd.metabolites.h_c: 1,
                              model_sucd.metabolites.nadh_c: -1,
                              succinyl_semialdehyde: -1, 
                              four_hydroxybutyrate: 1, #new one
                              model_sucd.metabolites.nad_c: 1
                             })
print(fhbd.build_reaction_string())
model_sucd.add_reactions([fhbd])
model_sucd.reactions.FHBD

nadh_c + succinyl_semialdehyde_c --> four_hydroxybutyrate_c + h_c + nad_c


Reaction identifier,FHBD
Name,
Memory address,0x02587739ed90
Stoichiometry,nadh_c + succinyl_semialdehyde_c --> four_hydroxybutyrate_c + h_c + nad_c Nicotinamide adenine dinucleotide - reduced + --> + H+ + Nicotinamide adenine dinucleotide
GPR,
Lower bound,0.0
Upper bound,1000.0


In [10]:
from cobra import Reaction, Metabolite
hbd_coa_t = Reaction('HBCT') #4-hydroxybutyryl-CoA transferase
four_hbcoa = Metabolite(id='four_hbcoa_c', compartment='c')
hbd_coa_t.add_metabolites({model_sucd.metabolites.accoa_c: -1,
                              model_sucd.metabolites.ac_c: 1,
                              four_hbcoa: 1, #new one
                              four_hydroxybutyrate: -1
                             })
print(hbd_coa_t.build_reaction_string())
model_sucd.add_reactions([hbd_coa_t])
model_sucd.reactions.HBCT

accoa_c + four_hydroxybutyrate_c --> ac_c + four_hbcoa_c


Reaction identifier,HBCT
Name,
Memory address,0x02587739eac0
Stoichiometry,accoa_c + four_hydroxybutyrate_c --> ac_c + four_hbcoa_c Acetyl-CoA + --> Acetate +
GPR,
Lower bound,0.0
Upper bound,1000.0


In [11]:
from cobra import Reaction, Metabolite
hbd_coa_r = Reaction('HBCR') #4-hydroxybutyryl-CoA reductase
four_hbaldehyde = Metabolite(id='four_hbaldehyde_c', compartment='c')
hbd_coa_r.add_metabolites({model_sucd.metabolites.h_c: 1,
                              model_sucd.metabolites.nadh_c: -1,
                              four_hbaldehyde: 1, #new one
                              four_hbcoa: -1,
                              model_sucd.metabolites.nad_c: 1,
                              model_sucd.metabolites.coa_c: 1
                             })
print(hbd_coa_r.build_reaction_string())
model_sucd.add_reactions([hbd_coa_r])
model_sucd.reactions.HBCR

four_hbcoa_c + nadh_c --> coa_c + four_hbaldehyde_c + h_c + nad_c


Reaction identifier,HBCR
Name,
Memory address,0x02587739ea60
Stoichiometry,four_hbcoa_c + nadh_c --> coa_c + four_hbaldehyde_c + h_c + nad_c + Nicotinamide adenine dinucleotide - reduced --> Coenzyme A + + H+ + Nicotinamide adenine dinucleotide
GPR,
Lower bound,0.0
Upper bound,1000.0


In [12]:
from cobra import Reaction, Metabolite
adh= Reaction('ADH') #alcohol dehydrogenase
bdo = Metabolite(id='bdo_c', compartment='c')
adh.add_metabolites({model_sucd.metabolites.h_c: 1,
                              model_sucd.metabolites.nadh_c: -1,
                              four_hbaldehyde: -1,
                              bdo: 1, #new one
                              model_sucd.metabolites.nad_c: 1,
                             })
print(adh.build_reaction_string())
model_sucd.add_reactions([adh])
model_sucd.reactions.ADH

four_hbaldehyde_c + nadh_c --> bdo_c + h_c + nad_c


Reaction identifier,ADH
Name,
Memory address,0x0258773a5be0
Stoichiometry,four_hbaldehyde_c + nadh_c --> bdo_c + h_c + nad_c + Nicotinamide adenine dinucleotide - reduced --> + H+ + Nicotinamide adenine dinucleotide
GPR,
Lower bound,0.0
Upper bound,1000.0


In [13]:
bdo_exchange = Reaction('EX_bdo')
bdo_exchange.add_metabolites({model_sucd.metabolites.bdo_c: -1})
model_sucd.add_reaction(bdo_exchange)
model_sucd.add_boundary(model_sucd.metabolites.bdo_c, type='demand')

Reaction identifier,DM_bdo_c
Name,demand
Memory address,0x0258773a50d0
Stoichiometry,bdo_c --> -->
GPR,
Lower bound,0
Upper bound,1000.0


In [14]:
with model_sucd:
    model_sucd.objective=model_sucd.reactions.ADH
    print(model_sucd.optimize().objective_value)

9.605894736842117


In [15]:
model_akg=model.copy()
model_akg

Name,iML1515
Memory address,0x0258773a5bb0
Number of metabolites,1877
Number of reactions,2712
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [16]:
from cobra import Reaction, Metabolite
oxodc = Reaction('OXODC') #2-oxoglutarate decarboxylase
succinyl_semialdehyde = Metabolite(id='succinyl_semialdehyde_c', compartment='c')

oxodc.add_metabolites({model_akg.metabolites.akg_c: -1,
                              succinyl_semialdehyde: 1, #new one
                              model_akg.metabolites.co2_c: 1
                             })
print(oxodc.build_reaction_string())
model_akg.add_reactions([oxodc])
model_akg.reactions.OXODC

akg_c --> co2_c + succinyl_semialdehyde_c


Reaction identifier,OXODC
Name,
Memory address,0x025877e898e0
Stoichiometry,akg_c --> co2_c + succinyl_semialdehyde_c 2-Oxoglutarate --> CO2 CO2 +
GPR,
Lower bound,0.0
Upper bound,1000.0


In [17]:
from cobra import Reaction, Metabolite
fhbd = Reaction('FHBD') #4-hydroxybutyrate dehydrogenase
four_hydroxybutyrate = Metabolite(id='four_hydroxybutyrate_c', compartment='c')

fhbd.add_metabolites({model_akg.metabolites.h_c: 1,
                              model_akg.metabolites.nadh_c: -1,
                              succinyl_semialdehyde: -1, 
                              four_hydroxybutyrate: 1, #new one
                              model_akg.metabolites.nad_c: 1
                             })
print(fhbd.build_reaction_string())
model_akg.add_reactions([fhbd])
model_akg.reactions.FHBD

nadh_c + succinyl_semialdehyde_c --> four_hydroxybutyrate_c + h_c + nad_c


Reaction identifier,FHBD
Name,
Memory address,0x025877e89a30
Stoichiometry,nadh_c + succinyl_semialdehyde_c --> four_hydroxybutyrate_c + h_c + nad_c Nicotinamide adenine dinucleotide - reduced + --> + H+ + Nicotinamide adenine dinucleotide
GPR,
Lower bound,0.0
Upper bound,1000.0


In [18]:
from cobra import Reaction, Metabolite
hbd_coa_t = Reaction('HBCT') #4-hydroxybutyryl-CoA transferase
four_hbcoa = Metabolite(id='four_hbcoa_c', compartment='c')
hbd_coa_t.add_metabolites({model_akg.metabolites.accoa_c: -1,
                              model_akg.metabolites.ac_c: 1,
                              four_hbcoa: 1, #new one
                              four_hydroxybutyrate: -1
                             })
print(hbd_coa_t.build_reaction_string())
model_akg.add_reactions([hbd_coa_t])
model_akg.reactions.HBCT

accoa_c + four_hydroxybutyrate_c --> ac_c + four_hbcoa_c


Reaction identifier,HBCT
Name,
Memory address,0x025877e8dd30
Stoichiometry,accoa_c + four_hydroxybutyrate_c --> ac_c + four_hbcoa_c Acetyl-CoA + --> Acetate +
GPR,
Lower bound,0.0
Upper bound,1000.0


In [19]:
from cobra import Reaction, Metabolite
hbd_coa_r = Reaction('HBCR') #4-hydroxybutyryl-CoA reductase
four_hbaldehyde = Metabolite(id='four_hbaldehyde_c', compartment='c')
hbd_coa_r.add_metabolites({model_akg.metabolites.h_c: 1,
                              model_akg.metabolites.nadh_c: -1,
                              four_hbaldehyde: 1, #new one
                              four_hbcoa: -1,
                              model_akg.metabolites.nad_c: 1,
                              model_akg.metabolites.coa_c: 1
                             })
print(hbd_coa_r.build_reaction_string())
model_akg.add_reactions([hbd_coa_r])
model_akg.reactions.HBCR

four_hbcoa_c + nadh_c --> coa_c + four_hbaldehyde_c + h_c + nad_c


Reaction identifier,HBCR
Name,
Memory address,0x025877e895e0
Stoichiometry,four_hbcoa_c + nadh_c --> coa_c + four_hbaldehyde_c + h_c + nad_c + Nicotinamide adenine dinucleotide - reduced --> Coenzyme A + + H+ + Nicotinamide adenine dinucleotide
GPR,
Lower bound,0.0
Upper bound,1000.0


In [20]:
from cobra import Reaction, Metabolite
adh= Reaction('ADH') #alcohol dehydrogenase
bdo = Metabolite(id='bdo_c', compartment='c')
adh.add_metabolites({model_akg.metabolites.h_c: 1,
                              model_akg.metabolites.nadh_c: -1,
                              four_hbaldehyde: -1,
                              bdo: 1, #new one
                              model_akg.metabolites.nad_c: 1,
                             })
print(adh.build_reaction_string())
model_akg.add_reactions([adh])
model_akg.reactions.ADH

four_hbaldehyde_c + nadh_c --> bdo_c + h_c + nad_c


Reaction identifier,ADH
Name,
Memory address,0x025877e8d670
Stoichiometry,four_hbaldehyde_c + nadh_c --> bdo_c + h_c + nad_c + Nicotinamide adenine dinucleotide - reduced --> + H+ + Nicotinamide adenine dinucleotide
GPR,
Lower bound,0.0
Upper bound,1000.0


In [21]:
bdo_exchange = Reaction('EX_bdo')
bdo_exchange.add_metabolites({model_akg.metabolites.bdo_c: -1})
model_akg.add_reaction(bdo_exchange)
model_akg.add_boundary(model_akg.metabolites.bdo_c, type='demand')

Reaction identifier,DM_bdo_c
Name,demand
Memory address,0x025877e8d910
Stoichiometry,bdo_c --> -->
GPR,
Lower bound,0
Upper bound,1000.0


In [22]:
with model_akg:
    model_akg.objective=model_akg.reactions.ADH
    print(model_akg.optimize().objective_value)

9.708085106382972


In [23]:
pip install cameo

Note: you may need to restart the kernel to use updated packages.


In [24]:
import cameo
from cameo import models
from cobra.io import read_sbml_model, write_sbml_model
from cobra import Metabolite, Reaction
from cobra.flux_analysis.phenotype_phase_plane import production_envelope
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [25]:
model_sucd.optimize()

,fluxes,reduced_costs
CYTDK2,0.00000,-7.273028e-17
XPPT,0.00000,-1.504671e-02
HXPRT,0.00000,-1.504671e-02
NDPK5,0.00000,-0.000000e+00
SHK3Dr,0.33424,0.000000e+00
...,...,...
HBCT,0.00000,0.000000e+00
HBCR,0.00000,0.000000e+00
ADH,0.00000,-1.170938e-17
EX_bdo,0.00000,0.000000e+00
